In [1]:
import pandas as pd
import json
import sys
import os
sys.path.append(os.path.abspath(''))

In [4]:
sentiment_file_path = '../../data/sentiment/sentiment_2023.csv'
sentiment_df = pd.read_csv(sentiment_file_path)
start_uid = sentiment_df['UID'].min()
last_uid = sentiment_df['UID'].max()

market_cap_file_path = '../../data/market_cap/market_cap_2023.csv'
market_cap_df = pd.read_csv(market_cap_file_path)

nasdaq_100_members_file_path = '../../data/nasdaq_100/nasdaq_100_members_2015_2024.json'
nasdaq_100_members = json.load(open(nasdaq_100_members_file_path))
nasdaq_100_members_2023 = nasdaq_100_members['2023']




## Preprocess Market Cap Data

In [3]:
market_cap_df.head()
market_cap_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33075 entries, 0 to 33074
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Date                  33075 non-null  object 
 1   Ticker                33075 non-null  object 
 2   market_cap(millions)  33075 non-null  float64
dtypes: float64(1), object(2)
memory usage: 775.3+ KB


In [4]:
print('filetering nasdaq 100 members')
print(f"before filtering: {len(market_cap_df['Ticker'].unique().tolist())}")
market_cap_df = market_cap_df[market_cap_df['Ticker'].isin(nasdaq_100_members_2023)]
print(f"after filtering: {len(market_cap_df['Ticker'].unique().tolist())}")

filetering nasdaq 100 members
before filtering: 133
after filtering: 102


In [5]:
# Calculate market cap weight
market_cap_df['market_cap_weight'] = market_cap_df['market_cap(millions)'] / market_cap_df.groupby('Date')['market_cap(millions)'].transform('sum')

In [6]:
market_cap_df.head()

,Date,Ticker,market_cap(millions),market_cap_weight
0,2023-12-29,VRSK,346.315948,0.001678
1,2023-12-29,MCHP,487.914381,0.002365
2,2023-12-29,LRCX,1032.274019,0.005003
4,2023-12-29,ADBE,2716.319800,0.013164
6,2023-12-29,MAR,662.302574,0.003210


## Preprocess Sentiment Data

In [7]:
sentiment_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63327 entries, 0 to 63326
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   UID        63327 non-null  int64 
 1   Date       63327 non-null  object
 2   Ticker     63327 non-null  object
 3   Sentiment  63327 non-null  object
 4   Reason     63327 non-null  object
dtypes: int64(1), object(4)
memory usage: 2.4+ MB


In [8]:
display(sentiment_df[sentiment_df['UID']==start_uid])
display(sentiment_df[sentiment_df['UID']==last_uid])

,UID,Date,Ticker,Sentiment,Reason
0,145799,2023-01-01,GOOG,Bullish,The article suggests Alphabet is well-position...


,UID,Date,Ticker,Sentiment,Reason
63326,209125,2023-12-31,AMD,Slightly Bullish,AMD rates highest using a Twin Momentum Invest...


In [9]:
sentiment_df.head()

,UID,Date,Ticker,Sentiment,Reason
0,145799,2023-01-01,GOOG,Bullish,The article suggests Alphabet is well-position...
1,145800,2023-01-01,MU,Bearish,The article discusses Micron continually revis...
2,145801,2023-01-01,ADBE,Neutral,The article presents a bull vs. bear case for ...
3,145802,2023-01-01,AAPL,Bullish,The article highlights Apple as Warren Buffett...
4,145803,2023-01-01,AAPL,Neutral,The article questions if Apple stock is a buy ...


In [10]:
sentiment_df['Sentiment'].unique()

array(['Bullish', 'Bearish', 'Neutral', 'Strongly Bullish',
       'Slightly Bearish', 'Strongly Bearish', 'Slightly Bullish'],
      dtype=object)

In [11]:
sentiment_mapping = {
    'Strongly Bearish': -3,
    'Bearish': -2,
    'Slightly Bearish': -1,
    'Neutral': 0,
    'Slightly Bullish': 1,
    'Bullish': 2,
    'Strongly Bullish': 3
}
sentiment_df['SentimentScore'] = sentiment_df['Sentiment'].map(sentiment_mapping)

# Convert 'Date' column to datetime objects to ensure correct grouping
sentiment_df['Date'] = pd.to_datetime(sentiment_df['Date'])

# Group by Date and Ticker, then calculate the mean of SentimentScore
aggregated_sentiment = sentiment_df.groupby(['Date', 'Ticker'])['SentimentScore'].mean().reset_index()

aggregated_sentiment.head()

,Date,Ticker,SentimentScore
0,2023-01-01,AAPL,1.0
1,2023-01-01,ADBE,0.0
2,2023-01-01,ASML,2.0
3,2023-01-01,CHTR,-1.0
4,2023-01-01,CMCSA,-2.0


## Map sentiment score to market cap weight

In [12]:
market_cap_df['Date'] = pd.to_datetime(market_cap_df['Date'])

# Merge the aggregated sentiment with market cap data
merged_df = pd.merge(aggregated_sentiment, market_cap_df, on=['Date', 'Ticker'], how='inner')


# Calculate the weighted sentiment for each ticker on each day
merged_df['WeightedSentiment'] = merged_df['SentimentScore'] * merged_df['market_cap_weight']

# Calculate the combined sentiment score for the Nasdaq 100 for each day
nasdaq_100_daily_sentiment = merged_df.groupby('Date')['WeightedSentiment'].sum().reset_index()

# Rename the resulting column for clarity
nasdaq_100_daily_sentiment.rename(columns={'WeightedSentiment': 'Nasdaq100_Weighted_Sentiment'}, inplace=True)


# Calculate the sum of 'market_cap_weight' from merged_df, grouped by 'Date'.
sentiment_coverage_by_market_cap = merged_df.groupby('Date')['market_cap_weight'].sum().reset_index()

# Rename the resulting column for clarity
sentiment_coverage_by_market_cap.rename(columns={'market_cap_weight': 'SentimentCoverageRatio'}, inplace=True)


nasdaq_100_daily_sentiment = pd.merge(nasdaq_100_daily_sentiment, sentiment_coverage_by_market_cap, on='Date', how='left')
nasdaq_100_daily_sentiment.head()

,Date,Nasdaq100_Weighted_Sentiment,SentimentCoverageRatio
0,2023-01-03,0.083663,0.591425
1,2023-01-04,0.190146,0.640182
2,2023-01-05,0.226121,0.624121
3,2023-01-06,0.560479,0.611186
4,2023-01-09,0.470395,0.616958


In [13]:
# Ensure the DataFrame is sorted by Date
nasdaq_100_daily_sentiment = nasdaq_100_daily_sentiment.sort_values(by='Date')

# Calculate rolling averages
nasdaq_100_daily_sentiment['3_day_rolling_sentiment'] = nasdaq_100_daily_sentiment['Nasdaq100_Weighted_Sentiment'].rolling(window=3, min_periods=1).mean()
nasdaq_100_daily_sentiment['5_day_rolling_sentiment'] = nasdaq_100_daily_sentiment['Nasdaq100_Weighted_Sentiment'].rolling(window=5, min_periods=1).mean()
nasdaq_100_daily_sentiment['7_day_rolling_sentiment'] = nasdaq_100_daily_sentiment['Nasdaq100_Weighted_Sentiment'].rolling(window=7, min_periods=1).mean()

# Display the DataFrame with the new rolling sentiment columns
nasdaq_100_daily_sentiment.head(10)

,Date,Nasdaq100_Weighted_Sentiment,SentimentCoverageRatio,3_day_rolling_sentiment,5_day_rolling_sentiment,7_day_rolling_sentiment
0,2023-01-03,0.083663,0.591425,0.083663,0.083663,0.083663
1,2023-01-04,0.190146,0.640182,0.136904,0.136904,0.136904
2,2023-01-05,0.226121,0.624121,0.166643,0.166643,0.166643
3,2023-01-06,0.560479,0.611186,0.325582,0.265102,0.265102
4,2023-01-09,0.470395,0.616958,0.418998,0.306161,0.306161
5,2023-01-10,0.219691,0.588551,0.416855,0.333366,0.291749
6,2023-01-11,0.310246,0.637961,0.333444,0.357386,0.294391
7,2023-01-12,0.359772,0.607683,0.296570,0.384117,0.333836
8,2023-01-13,0.242253,0.600935,0.304090,0.320471,0.341280
9,2023-01-17,0.264637,0.614203,0.288887,0.279320,0.346782


In [14]:
nasdaq_100_daily_sentiment.describe()

,Date,Nasdaq100_Weighted_Sentiment,SentimentCoverageRatio,3_day_rolling_sentiment,5_day_rolling_sentiment,7_day_rolling_sentiment
count,250,250.000000,250.000000,250.000000,250.000000,250.000000
mean,2023-07-02 09:18:43.200000,0.423663,0.680946,0.424004,0.424633,0.425547
min,2023-01-03 00:00:00,-0.062338,0.010738,0.007751,0.010435,0.010971
25%,2023-04-03 06:00:00,0.313791,0.682319,0.334202,0.334962,0.333029
50%,2023-07-04 00:00:00,0.443007,0.721416,0.447296,0.459755,0.459604
75%,2023-10-01 06:00:00,0.559397,0.736258,0.532043,0.523306,0.519553
max,2023-12-29 00:00:00,0.897642,0.768796,0.741362,0.680718,0.669083
std,NaN,0.191475,0.137157,0.152095,0.138994,0.129636


## Svae the data

In [15]:
nasdaq_100_daily_sentiment.to_csv('data/sentiment/nasdaq_100_daily_sentiment.csv', index=False)